# Video Frames (Images) to Velocity Fields with OpenPIV 
### Particle Image Velocimetry 

In [ ]:
from openpiv import tools, process, validation, filters, scaling 

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import imageio

import pandas as pd
import xarray as xr

In [ ]:
import openpiv.tools
import openpiv.process 
import openpiv.scaling 
import openpiv.validation 
import openpiv.filters
import glob

# Batch Processing

### check if pictures are in order, if not, sort them

In [ ]:
filepath = '/Users/.../OpenPIV/Frames/0726/Flight10_1440_Vpt4-5-6-5PostTypeA/DJI_0003_20180726_1440-Vpt6postTypeA_Frames'


files = glob.glob(filepath + "/*.jpg")
files

In [ ]:
def Frame_sort(x):
    return(x[-7:-4])

files_S =sorted(files, key = Frame_sort)
files_S

### Extract frame 0 and make a list of frames (will use this to save files based off of frame 0)

In [ ]:
frame0= files_S[0]
frame0 = int(frame0[-7:-4])
frame0

In [ ]:
FrameN = []
for i in range(len(files)):
    F1 = str(frame0 +i) #FOR ONE SECOND INTERVALS 
    FrameN.append(F1)
    i = i+1
FrameN

In [ ]:
#Change to desired output directory 
savedir = '/Users/Jasper/Volc_Research/OpenPIV/Site4/0726/F10/'

In [ ]:
#List is what the output file names will look like
i=0
for i in range(len(files_S)-1):
    print(savedir + 'Vel_Field_' + "%03d" %  i + '-' + "%03d" % (i+1) + '.csv' ) 

## Batch Process

In [ ]:
i=0
for i in range(len(files_S)-1):
    frame_a  = tools.imread(files_S[i])
    frame_b  = tools.imread(files_S[i+1])
    frame_a = (frame_a).astype(np.int32)
    frame_b = (frame_b).astype(np.int32)

    u, v, sig2noise = process.extended_search_area_piv( frame_a, frame_b, \
        window_size=64, overlap=32, dt=1, search_area_size=128, sig2noise_method='peak2peak' )
    x, y = process.get_coordinates( image_size=frame_a.shape, window_size=64, overlap=32 )
    u, v, mask = validation.sig2noise_val( u, v, sig2noise, threshold = 1.2 )
    u, v, mask = validation.global_val( u, v, (-1000, 2000), (-1000, 1000) )
    u, v = filters.replace_outliers( u, v, method='localmean', max_iter=10, kernel_size=2)
    x, y, u, v = scaling.uniform(x, y, u, v, scaling_factor = 1)
    tools.save(x, y, u, v, mask, savedir + 'Vel_Field_' + "%03d" %  i + '-' + "%03d" % (i+1) + '.csv')
    #tools.display_vector_field(savedir + 'Vel_Field_' + FrameN[i]+ '-' + FrameN[i+1] + '.csv', scale=400, width=0.001)
    i =i+1
    

In [ ]:
#does this look reasonable?
tools.display_vector_field(savedir + 'Vel_Field_000-001.csv' , scale=400, width=0.001)

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))
tools.display_vector_field(V_Fields[40], scale=400, width=0.001,ax=ax,on_img=True,image_name=files_S[10])

In [ ]:
#visual representation of what is being done (subtracting out mean of 'still area')
plt.plot(x[0,:],v[35,:].T)
plt.plot(x[0,:],np.ones_like(x[0,:])*3.5,'--')
plt.show()

### Load in V_Fields

In [ ]:
V_Fields = glob.glob(savedir + "*.csv") #This is the directory witht the CSV files


dfC = []
i=0
for i in range(len(V_Fields)):
    dfi = pd.read_csv(V_Fields[i], sep = '\t', header=None )
    dfi.columns = ('x','y','u', 'v', 'mask')
    #dfi.index.rename(['x','y'], inplace=True)
    dfi = dfi[~dfi.index.duplicated()]
    #dfi =dfi.to_xarray()
    file1 = V_Fields[i]
    sec = file1[len(file1)-11: len(file1)-4] # may need to be adjusted depending on file name length
    dfi['time']=sec
    dfC.append(dfi)
    i=i+1

### Choose 'still' regions for images 
Using the plotted Displayed Vector above you can get a pretty good idea of what is still

In [ ]:
#Overlap is important here, x and y are intervals of overlap size

def still(s1,s2,overlap):
    S1 = int(s1/overlap)*overlap
    S2 = int(s2/overlap)*overlap
    if S1<=overlap:
        S1 = S1 +overlap
    print(S1 , "-", S2)
    
    
def still_N(s1,s2,overlap):
    S1 = int(s1/overlap)-1
    S2 = int(s2/overlap)-1
    if S1<=overlap:
        S1 = 0
    print(S1 , "-", S2)

In [ ]:
#Visually the 'still' area for the image above can be aproxiamated by 
#x = 0-500, 3712-3840  y = 0 - 2160
print("X still ranges: ")
print(still(0,200,32), still(3700,3840,32))

print("Y still ranges: ")
print(still(0,2160,32))

In [ ]:
print("X still ranges: ")
print(still_N(0,192,32), still_N(3700,3840,32))

print("Y still ranges: ")
print(still_N(0,2160,32))

#Need to fix that function, but 
# X range: 32 - 480, 3680 - 3808;
# Y range: 32 - 2112

### Compute V mean and U mean for still regions

In [ ]:
vmean1 = dfC[40].loc[0:3].v.mean()
vmean2 = dfC[40].loc[114:119].v.mean() # in this case I choose 2 regions, but you can choose from 1 or more
vmean = (vmean1+vmean2)/2
vmean 

In [ ]:
umean1 = dfC[40].loc[0:3].u.mean()
umean2 = dfC[40].loc[114:119].u.mean() # in this case I choose 2 regions, but you can choose from 1 or more
umean = (umean1+umean2)/2
umean

In [ ]:
vmean1

### Save New Results 

In [ ]:
tools.save(x, y, u-umean1 -2, v-vmean1+1, mask, V_Fields[40] )
fig,ax = plt.subplots(figsize=(20,20))
tools.display_vector_field(V_Fields[40], scale=400, width=0.001,ax=ax,on_img=True,
                           image_name='/Users/Jasper/Volc_Research/OpenPIV/F10_Clip/f10_CLIP/DJI_0003_20180726_1440-Vpt6postTypeA-Copy1.MOV_115.jpg')

# Now Batch Process the corrected fields

For Site 4, this is the approximate still region

In [ ]:

V_Fields = glob.glob(savedir + "*.csv") #This is the directory with the CSV files


dfC = []
i=0
for i in range(len(V_Fields)):
    dfi = pd.read_csv(V_Fields[i], sep = '\t', header=None )
    dfi.columns = ('x','y','u', 'v', 'mask')
    #dfi.index.rename(['x','y'], inplace=True)
    dfi = dfi[~dfi.index.duplicated()]
    #dfi =dfi.to_xarray()
    file1 = V_Fields[i]
    sec = file1[len(file1)-11: len(file1)-4] # may need to be adjusted depending on file name length
    dfi['time']=sec
    dfC.append(dfi)
    i=i+1

In [ ]:
print("X still ranges: ")
print(still_N(0,500,32), still_N(3700,3840,32))

print("Y still ranges: ")
print(still_N(0,2160,32))

In [ ]:
#Display incorrected vector fields

V_Fields = glob.glob('/Users/Jasper/Volc_Research/OpenPIV/Subtract_test/' + "*.csv")

i = 0


for i in range(len(V_Fields)):

    tools.display_vector_field(V_Fields[i], scale=400, width=0.001)
     
    i+i+1

In [ ]:
#new save dir for corrected
#savedir2 = '/Users/Jasper/Volc_Research/OpenPIV/Site4/0726/F09_Cor/'
#V_Fields_Cor = glob.glob(savedir2 + "*.csv") #This is the directory with the CSV files


i = 0

for i in range(len(V_Fields)):
   
    vmean1 = dfC[i].loc[0:14].v.mean()
    vmean2 = dfC[i].loc[114:119].v.mean() # in this case I choose 2 regions, but you can choose from 1 or more
    vmean = (vmean1+vmean2)/2
    
    umean1 = dfC[i].loc[0:14].u.mean()
    umean2 = dfC[i].loc[114:119].u.mean() # in this case I choose 2 regions, but you can choose from 1 or more
    umean = (umean1+umean2)/2
    
    tools.save(x, y, u-umean, v-vmean, mask, V_Fields[i] +'C.csv')
    tools.display_vector_field(V_Fields[i], scale=400, width=0.001)
     
    i+i+1

### Check to see if results are Reasonable
still regions should have very little to no velocity

In [ ]:
tools.display_vector_field(V_Fields[119], scale=400, width=0.001)

In [ ]:
i=0
while(i<=len(files_S)):
    fig,ax = plt.subplots(1,2,figsize=(12,10))
    ax[0].imshow(frame_a,cmap=plt.cm.magma)
    ax[1].imshow(frame_b,cmap=plt.cm.magma)
    i = i+1